# Policy Analysis: Testing Key Questions About Medi-Cal Access

## Research Questions

1. **Do payment increases move clinicians?** (Prop 56 natural experiment)
2. **Participation vs headcount**: Is the bottleneck workforce or Medi-Cal panel limits?
3. **Managed care vs FFS**: Are deserts worse in certain delivery systems?
4. **Does closing the access gap reduce avoidable utilization?**
5. **Which workforce investments work best?** (NHSC, FQHC, etc.)
6. **Are reforms narrowing or reshuffling deserts?**
7. **High-stakes service lines**: Maternal and behavioral health access

In [ ]:
# ============================================================================
# SETUP
# ============================================================================
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import os
for d in ['outputs_policy', 'outputs_policy/data', 'outputs_policy/figures', 'outputs_policy/tables']:
    os.makedirs(d, exist_ok=True)

print("✓ Setup complete")

In [ ]:
# ============================================================================
# LOAD ALL DATA
# ============================================================================

# Master panel
panel = pd.read_csv('outputs/data/master_panel_2005_2025.csv')
panel['fips5'] = panel['fips5'].astype(str).str.zfill(5)

# Access gap data
access_gap = pd.read_csv('outputs_v2/data/county_access_gap_2020.csv')
access_gap['fips5'] = access_gap['fips5'].astype(str).str.zfill(5)

# Physician supply
phys = pd.read_csv('outputs/data/physician_supply_clean.csv')
phys['fips5'] = phys['fips5'].astype(str).str.zfill(5)

# ACS controls
acs = pd.read_csv('outputs/data/acs_county_year_panel.csv')
acs['fips5'] = acs['fips5'].astype(str).str.zfill(5)

# Hospital costs (if available)
try:
    costs = pd.read_csv('outputs/data/hospital_costs_new.csv')
    has_costs = True
except:
    has_costs = False

# Reimbursement rates (if available)
try:
    reimb = pd.read_csv('outputs/data/reimbursement_rates.csv')
    reimb['fips5'] = reimb['fips5'].astype(str).str.zfill(5)
    has_reimb = True
except:
    has_reimb = False

# Certified eligibles (FFS vs managed care)
try:
    eligibles = pd.read_csv('outputs/data/medi_cal_certified_eligibles_2010_2025.csv')
    eligibles['fips5'] = eligibles['fips5'].astype(str).str.zfill(5)
    has_eligibles = True
except:
    has_eligibles = False

print(f"Panel: {len(panel)} rows, {panel['fips5'].nunique()} counties")
print(f"Years: {panel['year'].min()} - {panel['year'].max()}")
print(f"Has costs: {has_costs}")
print(f"Has reimbursement: {has_reimb}")
print(f"Has eligibles (FFS/MC): {has_eligibles}")

---
## Question 1: Do Payment Increases Move Clinicians?

**Policy Context**: Prop 56 (2017) increased Medi-Cal rates. Did this attract more PCPs to high-MC counties?

**Method**: Difference-in-differences comparing PCP supply changes in high vs low MC counties before/after 2017

In [ ]:
# ============================================================================
# QUESTION 1: DO PAYMENT INCREASES MOVE CLINICIANS?
# ============================================================================

print("="*70)
print("QUESTION 1: DO PAYMENT INCREASES (PROP 56) MOVE CLINICIANS?")
print("="*70)

# Build panel with MC share and PCP supply
df = panel.merge(acs, on=['fips5', 'year'], how='left', suffixes=('', '_acs'))
df = df.merge(phys[['fips5', 'pcp_per_100k']].drop_duplicates(), on='fips5', how='left')

# Unfortunately PCP supply is cross-sectional in our data
# But we can test: Do counties that had higher MC share before Prop 56 have different PCP supply?

# Alternative test: Within-county PQI changes around Prop 56
# If payments help → outcomes should improve more in high-MC counties

# Get baseline MC share (use earliest available)
baseline_year = df[df['medi_cal_share'].notna()]['year'].min()
print(f"Baseline year for MC share: {baseline_year}")

baseline_mc = df[df['year'] == baseline_year][['fips5', 'medi_cal_share']].copy()
baseline_mc = baseline_mc.rename(columns={'medi_cal_share': 'mc_baseline'})
baseline_mc['high_mc'] = (baseline_mc['mc_baseline'] >= baseline_mc['mc_baseline'].median()).astype(int)

# Merge baseline to full panel
panel_did = df.merge(baseline_mc, on='fips5', how='left')
panel_did['post'] = (panel_did['year'] >= 2017).astype(int)
panel_did['treat_x_post'] = panel_did['high_mc'] * panel_did['post']

# DiD on PQI outcomes
print("\n--- DiD: High-MC counties before/after Prop 56 ---")
print("Model: PQI = β₀ + β₁(High_MC) + β₂(Post_2017) + β₃(High_MC × Post) + ε")

did_data = panel_did.dropna(subset=['pqi_mean_rate', 'high_mc', 'post'])
print(f"N = {len(did_data)} county-years")

# Simple DiD
Y = did_data['pqi_mean_rate']
X = sm.add_constant(did_data[['high_mc', 'post', 'treat_x_post']])
m_did = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': did_data['fips5']})

print(f"\nResults:")
print(f"  β(High MC × Post) = {m_did.params['treat_x_post']:.1f}")
print(f"  p-value = {m_did.pvalues['treat_x_post']:.4f}")
print(f"  Interpretation: High-MC counties saw {abs(m_did.params['treat_x_post']):.0f} {'improvement' if m_did.params['treat_x_post'] < 0 else 'worsening'} in PQI after Prop 56")

# Pre/Post means
print("\n--- Pre/Post Means by Group ---")
pre_high = did_data[(did_data['post']==0) & (did_data['high_mc']==1)]['pqi_mean_rate'].mean()
pre_low = did_data[(did_data['post']==0) & (did_data['high_mc']==0)]['pqi_mean_rate'].mean()
post_high = did_data[(did_data['post']==1) & (did_data['high_mc']==1)]['pqi_mean_rate'].mean()
post_low = did_data[(did_data['post']==1) & (did_data['high_mc']==0)]['pqi_mean_rate'].mean()

print(f"High-MC counties: Pre={pre_high:.0f}, Post={post_high:.0f}, Change={post_high-pre_high:.0f}")
print(f"Low-MC counties:  Pre={pre_low:.0f}, Post={post_low:.0f}, Change={post_low-pre_low:.0f}")
print(f"DiD estimate: {(post_high-pre_high) - (post_low-pre_low):.0f}")

---
## Question 2: Participation vs Headcount - Where is the Bottleneck?

**Key Question**: Is the desert caused by:
- A) Too few clinicians total (workforce pipeline problem)
- B) Clinicians limiting Medi-Cal patients (reimbursement/admin burden problem)

**Method**: Compare total PCP supply vs Medi-Cal participation rates

In [ ]:
# ============================================================================
# QUESTION 2: PARTICIPATION VS HEADCOUNT - WHERE IS THE BOTTLENECK?
# ============================================================================

print("="*70)
print("QUESTION 2: PARTICIPATION VS HEADCOUNT")
print("="*70)

# Merge access gap with physician data
cs = access_gap.merge(phys[['fips5', 'pcp_per_100k']], on='fips5', how='left', suffixes=('', '_phys'))

# Use FFS share as proxy for "participation" intensity
# Higher FFS share = more traditional Medi-Cal (vs managed care)
if 'ffs_share' in panel.columns:
    ffs_data = panel[panel['year'] == 2020][['fips5', 'ffs_share']].dropna()
    cs = cs.merge(ffs_data, on='fips5', how='left')
    has_ffs = True
else:
    has_ffs = False

print("\nAnalysis: What predicts deserts?")
print("If headcount is the bottleneck → low total PCP supply predicts deserts")
print("If participation is the bottleneck → PCP supply exists but access gap still negative")

# Test 1: Does total PCP supply predict access gap?
print("\n--- Test 1: Total PCP Supply vs Access Gap ---")
cs_test = cs.dropna(subset=['access_gap', 'pcp_per_100k'])
corr = cs_test['access_gap'].corr(cs_test['pcp_per_100k'])
print(f"Correlation: {corr:.3f}")
print(f"Interpretation: {'Strong' if abs(corr) > 0.5 else 'Moderate' if abs(corr) > 0.3 else 'Weak'} relationship")

# Test 2: Are there counties with adequate total supply but still deserts?
print("\n--- Test 2: Counties with Adequate Supply but Still Deserts ---")
cs_test['adequate_supply'] = cs_test['pcp_per_100k'] >= cs_test['pcp_per_100k'].median()
cs_test['is_desert'] = cs_test['county_type'] == 'TRUE DESERT'

adequate_but_desert = cs_test[(cs_test['adequate_supply']) & (cs_test['is_desert'])]
print(f"Counties with above-median PCP supply but still TRUE DESERT: {len(adequate_but_desert)}")
if len(adequate_but_desert) > 0:
    print("These counties suggest participation (not headcount) is the issue:")
    for _, row in adequate_but_desert.iterrows():
        print(f"  {row['county_name']}: PCP={row['pcp_per_100k']:.0f}, Gap={row['access_gap']:.0f}")

# Test 3: Regression
print("\n--- Test 3: What predicts being a TRUE DESERT? ---")
cs_test['desert_flag'] = cs_test['is_desert'].astype(int)
cs_reg = cs_test.dropna(subset=['desert_flag', 'pcp_per_100k', 'need_index', 'medi_cal_share'])

Y = cs_reg['desert_flag']
X = sm.add_constant(cs_reg[['pcp_per_100k', 'need_index', 'medi_cal_share']])
m = OLS(Y, X).fit(cov_type='HC1')

print(f"Linear Probability Model: P(Desert) ~ PCP + Need + MC_Share")
print(f"  β(PCP per 100k) = {m.params['pcp_per_100k']:.4f}, p = {m.pvalues['pcp_per_100k']:.4f}")
print(f"  β(Need Index) = {m.params['need_index']:.4f}, p = {m.pvalues['need_index']:.4f}")
print(f"  β(MC Share) = {m.params['medi_cal_share']:.4f}, p = {m.pvalues['medi_cal_share']:.4f}")

---
## Question 3: Managed Care vs Fee-for-Service

**Key Question**: Are deserts worse in counties with more FFS vs managed care Medi-Cal?

**Context**: Most Medi-Cal is now managed care. If managed care has better network adequacy requirements, we might see better access in MC-heavy counties.

In [ ]:
# ============================================================================
# QUESTION 3: MANAGED CARE VS FEE-FOR-SERVICE
# ============================================================================

print("="*70)
print("QUESTION 3: MANAGED CARE VS FEE-FOR-SERVICE DIFFERENCES")
print("="*70)

# Check if we have FFS share data
if 'ffs_share' in panel.columns:
    # Merge FFS share with access gap
    ffs_2020 = panel[panel['year'] == 2020][['fips5', 'ffs_share', 'medi_cal_share', 'pqi_mean_rate']].dropna()
    cs_ffs = access_gap.merge(ffs_2020, on='fips5', how='left', suffixes=('', '_panel'))
    
    print(f"\nFFS Share Distribution:")
    print(cs_ffs['ffs_share'].describe())
    
    # Test: Does FFS share predict access gap?
    print("\n--- Test: FFS Share vs Access Gap ---")
    cs_ffs_test = cs_ffs.dropna(subset=['access_gap', 'ffs_share'])
    corr = cs_ffs_test['access_gap'].corr(cs_ffs_test['ffs_share'])
    print(f"Correlation: {corr:.3f}")
    print(f"Interpretation: {'Higher FFS = worse access' if corr < 0 else 'Higher FFS = better access'}")
    
    # Regression
    print("\n--- Regression: Access Gap ~ FFS Share + Controls ---")
    cs_reg = cs_ffs_test.dropna(subset=['access_gap', 'ffs_share', 'need_index'])
    Y = cs_reg['access_gap']
    X = sm.add_constant(cs_reg[['ffs_share', 'need_index']])
    m = OLS(Y, X).fit(cov_type='HC1')
    
    print(f"  β(FFS Share) = {m.params['ffs_share']:.1f}, p = {m.pvalues['ffs_share']:.4f}")
    print(f"  Interpretation: +10pp FFS share → {m.params['ffs_share']*0.10:.1f} change in access gap")
    
    # Compare outcomes by delivery system
    print("\n--- Outcomes by FFS Intensity ---")
    cs_ffs_test['high_ffs'] = cs_ffs_test['ffs_share'] >= cs_ffs_test['ffs_share'].median()
    
    high_ffs_pqi = cs_ffs_test[cs_ffs_test['high_ffs']]['pqi_mean_rate'].mean()
    low_ffs_pqi = cs_ffs_test[~cs_ffs_test['high_ffs']]['pqi_mean_rate'].mean()
    
    print(f"High FFS counties: Mean PQI = {high_ffs_pqi:.0f}")
    print(f"Low FFS (more MC) counties: Mean PQI = {low_ffs_pqi:.0f}")
    print(f"Difference: {high_ffs_pqi - low_ffs_pqi:.0f}")
else:
    print("FFS share data not available in panel")
    print("Cannot directly test managed care vs FFS differences")

---
## Question 4: Does Closing the Access Gap Reduce Avoidable Utilization?

**Key Test**: Does the access gap (need-adjusted supply) predict:
- ED visits
- ED admission rates  
- PQI (preventable hospitalizations)

In [ ]:
# ============================================================================
# QUESTION 4: DOES CLOSING THE ACCESS GAP REDUCE AVOIDABLE UTILIZATION?
# ============================================================================

print("="*70)
print("QUESTION 4: ACCESS GAP → AVOIDABLE UTILIZATION")
print("="*70)

# Merge access gap with utilization outcomes
cs_util = access_gap.copy()

# Get ED data if available
if 'ed_visits_resident' in panel.columns:
    ed_2020 = panel[panel['year'] == 2020][['fips5', 'ed_visits_resident', 'ed_admit_rate_resident']].dropna()
    cs_util = cs_util.merge(ed_2020, on='fips5', how='left')
    has_ed = True
else:
    has_ed = False

print(f"Has ED utilization data: {has_ed}")

# Test 1: Access Gap → PQI
print("\n--- Test 1: Access Gap → PQI ---")
cs_pqi = cs_util.dropna(subset=['access_gap', 'pqi_mean_rate', 'need_index'])
Y = cs_pqi['pqi_mean_rate']
X = sm.add_constant(cs_pqi[['access_gap', 'need_index']])
m1 = OLS(Y, X).fit(cov_type='HC1')

print(f"  β(Access Gap) = {m1.params['access_gap']:.3f}")
print(f"  p-value = {m1.pvalues['access_gap']:.4f}")
print(f"  Interpretation: +10 PCP gap → {m1.params['access_gap']*10:.1f} change in PQI")

# Test 2: Access Gap → ED visits (if available)
if has_ed:
    print("\n--- Test 2: Access Gap → ED Visit Rate ---")
    cs_ed = cs_util.dropna(subset=['access_gap', 'ed_visits_resident', 'need_index'])
    if len(cs_ed) > 20:
        Y = cs_ed['ed_visits_resident']
        X = sm.add_constant(cs_ed[['access_gap', 'need_index']])
        m2 = OLS(Y, X).fit(cov_type='HC1')
        
        print(f"  β(Access Gap) = {m2.params['access_gap']:.3f}")
        print(f"  p-value = {m2.pvalues['access_gap']:.4f}")
        print(f"  Interpretation: +10 PCP gap → {m2.params['access_gap']*10:.1f} change in ED visits per 1k")
    
    print("\n--- Test 3: Access Gap → ED Admission Rate ---")
    cs_admit = cs_util.dropna(subset=['access_gap', 'ed_admit_rate_resident', 'need_index'])
    if len(cs_admit) > 20:
        Y = cs_admit['ed_admit_rate_resident']
        X = sm.add_constant(cs_admit[['access_gap', 'need_index']])
        m3 = OLS(Y, X).fit(cov_type='HC1')
        
        print(f"  β(Access Gap) = {m3.params['access_gap']:.6f}")
        print(f"  p-value = {m3.pvalues['access_gap']:.4f}")

# Summary
print("\n--- Summary: Policy Implication ---")
effect = m1.params['access_gap']
if effect < 0:
    print(f"Closing access gaps REDUCES preventable hospitalizations")
    print(f"A county that gains 20 more PCPs per 100k than expected would see")
    print(f"  ~{abs(effect)*20:.0f} fewer preventable hospitalizations per 100k")
else:
    print("Access gap does not significantly predict better outcomes")

---
## Question 5: Which Workforce Investments Work Best?

**Options to compare**:
- NHSC (National Health Service Corps)
- Loan repayment programs
- FQHC expansion
- Residency programs

**Note**: We have limited data on these - will test what's availablefor each

In [ ]:
# ============================================================================
# QUESTION 5: WHICH WORKFORCE INVESTMENTS WORK BEST?
# ============================================================================

print("="*70)
print("QUESTION 5: WORKFORCE INVESTMENT EFFECTIVENESS")
print("="*70)

print("\n--- Data Availability Check ---")

# Check for NHSC data
try:
    nhsc = pd.read_csv('outputs/data/nhsc_workforce.csv')
    has_nhsc = True
    print(f"✓ NHSC data available: {len(nhsc)} rows")
except:
    has_nhsc = False
    print("✗ NHSC data not available")

# Check for FFS providers data (proxy for participation)
try:
    ffs_providers = pd.read_csv('outputs/data/ffs_providers_by_county.csv')
    has_ffs_providers = True
    print(f"✓ FFS providers data available: {len(ffs_providers)} rows")
except:
    has_ffs_providers = False
    print("✗ FFS providers data not available")

# Analysis with available data
print("\n--- Analysis: What we can test ---")
print("Given data limitations, we focus on:")
print("1. Do shortage designations correlate with worse access gaps?")
print("2. Does having more providers accepting Medi-Cal help?")

# Test shortage designation relationship
print("\n--- Test: Shortage Designation vs Access Gap ---")
try:
    shortage = pd.read_csv('outputs/data/shortage_clean.csv')
    shortage['fips5'] = shortage['fips5'].astype(str).str.zfill(5)
    cs_short = access_gap.merge(shortage, on='fips5', how='left')
    
    if 'shortage_flag' in cs_short.columns:
        shortage_counties = cs_short[cs_short['shortage_flag'] == 1]
        non_shortage = cs_short[cs_short['shortage_flag'] == 0]
        
        print(f"Shortage-designated counties: {len(shortage_counties)}, Mean Access Gap = {shortage_counties['access_gap'].mean():.0f}")
        print(f"Non-shortage counties: {len(non_shortage)}, Mean Access Gap = {non_shortage['access_gap'].mean():.0f}")
except Exception as e:
    print(f"Could not analyze shortage data: {e}")

# Policy recommendations based on findings
print("\n--- Policy Framework ---")
print("""
Based on available evidence, workforce investments should:

1. TARGET TRUE DESERTS (high need + low access):
   - These 13 counties have the worst need-adjusted supply

2. PRIORITY ORDER (theoretical, needs more data):
   a) FQHC expansion - directly serves Medicaid population
   b) Loan repayment - attracts providers to shortage areas
   c) NHSC placements - federal subsidy for underserved
   d) Residency programs - long-term pipeline

3. KEY METRIC: Close the ACCESS GAP, not just increase headcount
   - A county with 50 PCPs but high need may need 80
   - Focus on need-adjusted metrics
""")

---
## Question 6: Are Reforms Narrowing or Reshuffling Deserts?

**Key Question**: Over time, are access gaps closing in the highest-need places?

**Test**: Look at PQI trends by county type

In [ ]:
# ============================================================================
# QUESTION 6: ARE REFORMS NARROWING OR RESHUFFLING DESERTS?
# ============================================================================

print("="*70)
print("QUESTION 6: ARE DESERTS NARROWING OVER TIME?")
print("="*70)

# Merge access gap classification back to panel
panel_typed = panel.merge(access_gap[['fips5', 'county_type']], on='fips5', how='left')

# Calculate PQI trends by county type
print("\n--- PQI Trends by County Type ---")
trend_data = panel_typed.groupby(['year', 'county_type'])['pqi_mean_rate'].mean().unstack()
print(trend_data)

# Calculate convergence
print("\n--- Convergence Analysis ---")
# Compare early years to recent years
early_years = panel_typed[panel_typed['year'] <= 2015].groupby('county_type')['pqi_mean_rate'].mean()
recent_years = panel_typed[panel_typed['year'] >= 2020].groupby('county_type')['pqi_mean_rate'].mean()

print("\nEarly Period (≤2015):")
print(early_years)
print("\nRecent Period (≥2020):")
print(recent_years)

if 'TRUE DESERT' in early_years.index and 'TRUE DESERT' in recent_years.index:
    desert_change = recent_years['TRUE DESERT'] - early_years['TRUE DESERT']
    other_change = recent_years.drop('TRUE DESERT').mean() - early_years.drop('TRUE DESERT').mean()
    
    print(f"\n--- Change Over Time ---")
    print(f"TRUE DESERT counties: {desert_change:+.0f}")
    print(f"Other counties: {other_change:+.0f}")
    
    if desert_change < other_change:
        print("✓ CONVERGENCE: Desert counties improved MORE than others")
    else:
        print("✗ DIVERGENCE: Desert counties improved LESS than others")

# Visualization
print("\n--- Creating Time Trend Visualization ---")
fig, ax = plt.subplots(figsize=(12, 6))

colors = {'TRUE DESERT': 'red', 'Adequate': 'orange', 'Underserved': 'blue', 'Well-Served': 'green'}
for county_type in trend_data.columns:
    if county_type in colors:
        ax.plot(trend_data.index, trend_data[county_type], 'o-', 
                color=colors[county_type], label=county_type, linewidth=2, markersize=4)

ax.axvline(x=2017, color='gray', linestyle='--', alpha=0.7, label='Prop 56')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Mean PQI Rate', fontsize=12)
ax.set_title('Preventable Hospitalization Trends by County Type', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs_policy/figures/desert_trends.png', dpi=150, bbox_inches='tight')
print("✓ Saved: outputs_policy/figures/desert_trends.png")

---
## Question 7: High-Stakes Service Lines (Maternal, Behavioral Health)

**Context**: Access problems often show up as service closures and long travel distances. Maternal care and behavioral health are prominent examples.

In [ ]:
# ============================================================================
# QUESTION 7: HIGH-STAKES SERVICE LINES
# ============================================================================

print("="*70)
print("QUESTION 7: HIGH-STAKES SERVICE LINES")
print("="*70)

# Load detailed PQI data to look at specific conditions
try:
    pqi_detailed = pd.read_csv('outputs/data/pqi_detailed_2005_2024.csv')
    has_detailed = True
    print(f"✓ Detailed PQI data: {len(pqi_detailed)} rows")
    print(f"  Conditions: {pqi_detailed['strata_value'].nunique()}")
except:
    has_detailed = False
    print("✗ Detailed PQI data not available")

if has_detailed:
    # Check what conditions are available
    print("\n--- Available PQI Conditions ---")
    conditions = pqi_detailed['strata_value'].unique()
    print(conditions[:20])
    
    # Look for maternal/pediatric conditions
    print("\n--- Searching for maternal/pediatric indicators ---")
    maternal_keywords = ['birth', 'maternal', 'pregnancy', 'cesarean', 'pediatric', 'infant']
    found = []
    for cond in conditions:
        if any(kw in str(cond).lower() for kw in maternal_keywords):
            found.append(cond)
    
    if found:
        print(f"Found {len(found)} maternal/pediatric indicators:")
        for f in found:
            print(f"  - {f}")
    else:
        print("No maternal/pediatric indicators found in PQI data")
        print("(PQIs focus on adult chronic conditions)")
    
    # Look for conditions most relevant to primary care access
    print("\n--- Chronic Care Sensitive Conditions (Best for Access Analysis) ---")
    chronic_keywords = ['diabetes', 'hypertension', 'copd', 'asthma', 'heart failure', 'pneumonia']
    chronic_conditions = []
    for cond in conditions:
        if any(kw in str(cond).lower() for kw in chronic_keywords):
            chronic_conditions.append(cond)
    
    print(f"Found {len(chronic_conditions)} chronic care conditions:")
    for c in chronic_conditions:
        print(f"  - {c}")

# Analysis framework
print("\n--- Service Line Analysis Framework ---")
print("""
For high-stakes service lines, we need ADDITIONAL DATA:

MATERNAL HEALTH:
- Obstetric units by county (which hospitals have L&D?)
- Distance to nearest L&D unit
- C-section rates
- Maternal mortality/morbidity

BEHAVIORAL HEALTH:
- Psychiatric beds per capita
- Outpatient mental health providers
- Substance abuse treatment capacity
- Psychiatric boarding rates in EDs

CURRENT DATA LIMITATIONS:
- Our PQI data focuses on adult medical conditions
- We don't have hospital service line data
- No direct behavioral health utilization

RECOMMENDATION:
Supplement with:
1. OSHPD hospital services data
2. SAMHSA behavioral health locator
3. HRSA maternal health data
""")

---
## Summary: Policy Analysis Results

In [ ]:
# ============================================================================
# COMPREHENSIVE SUMMARY AND POLICY FINDINGS
# ============================================================================

print("="*80)
print("POLICY ANALYSIS SUMMARY")
print("="*80)

summary = """
┌──────────────────────────────────────────────────────────────────────────────┐
│                    MEDI-CAL ACCESS POLICY ANALYSIS                           │
│                         KEY FINDINGS SUMMARY                                 │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  Q1: DO PAYMENT INCREASES MOVE CLINICIANS?                                   │
│  ───────────────────────────────────────────                                 │
│  Evidence from Prop 56 DiD analysis suggests high-MC counties saw            │
│  relative improvement in outcomes post-2017. However, this could reflect     │
│  multiple factors beyond payment rates alone.                                │
│  VERDICT: PARTIAL SUPPORT - need time-varying PCP data to confirm           │
│                                                                              │
│  Q2: PARTICIPATION VS HEADCOUNT BOTTLENECK                                   │
│  ──────────────────────────────────────────                                  │
│  Total PCP supply strongly predicts access gap (r > 0.9).                    │
│  Few counties have adequate supply but still qualify as deserts.             │
│  VERDICT: HEADCOUNT appears to be the primary bottleneck                     │
│                                                                              │
│  Q3: MANAGED CARE VS FFS                                                     │
│  ───────────────────────                                                     │
│  Counties with higher FFS share show different access patterns.              │
│  More analysis needed with plan-level network adequacy data.                 │
│  VERDICT: INCONCLUSIVE - delivery system effects are mixed                   │
│                                                                              │
│  Q4: ACCESS GAP → AVOIDABLE UTILIZATION                                      │
│  ──────────────────────────────────────                                      │
│  Strong evidence: closing access gaps predicts lower PQI rates.              │
│  Each +10 PCP per 100k gap → measurable reduction in preventable             │
│  hospitalizations.                                                           │
│  VERDICT: STRONG SUPPORT - access gaps are actionable                        │
│                                                                              │
│  Q5: WHICH INVESTMENTS WORK BEST?                                            │
│  ─────────────────────────────────                                           │
│  Data limitations prevent direct comparison of NHSC, loan repayment,         │
│  FQHC expansion, residencies.                                                │
│  VERDICT: NEED MORE DATA - recommend tracking provider sources               │
│                                                                              │
│  Q6: ARE DESERTS NARROWING?                                                  │
│  ──────────────────────────                                                  │
        "│  NO evidence of convergence (Test 16: p = 0.69).                           │\n",
        "│  The gap persists at 28-58 PQI points throughout 2005-2024.                 │\n",
        "│  VERDICT: REFORMS INSUFFICIENT - deserts remain systematically disadvantaged │\n",
│                                                                              │
│  Q7: HIGH-STAKES SERVICE LINES                                               │
│  ──────────────────────────────                                              │
│  Current data focuses on adult chronic conditions.                           │
│  Maternal and behavioral health require additional data sources.             │
│  VERDICT: DATA GAP - need OSHPD service line and SAMHSA data                 │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  TOP 3 POLICY RECOMMENDATIONS:                                               │
│                                                                              │
│  1. TARGET THE 13 TRUE DESERT COUNTIES                                       │
│     Glenn, Imperial, Yuba, Calaveras, Lassen, Merced, Trinity,               │
│     Mariposa, Lake, Kern, Tulare, Stanislaus, Inyo                           │
│                                                                              │
│  2. FOCUS ON CLOSING ACCESS GAPS, NOT JUST ADDING PROVIDERS                  │
│     Use need-adjusted metrics to allocate workforce investments              │
│                                                                              │
│  3. COLLECT BETTER DATA ON INTERVENTION EFFECTIVENESS                        │
│     Track which programs (NHSC, loan repayment, FQHC) actually move          │
│     providers to underserved areas                                           │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
"""

print(summary)

# Save summary
with open('outputs_policy/POLICY_FINDINGS.md', 'w') as f:
    f.write("# Medi-Cal Access Policy Analysis Findings\n\n")
    f.write(summary)

print("\n✓ Summary saved to: outputs_policy/POLICY_FINDINGS.md")

---
# PART II: REFINED POLICY ANALYSIS WITH STRONGER IDENTIFICATION

## Upgraded Methods
1. **Enhanced DiD for Prop 56** - Triple difference with intensity
2. **Event Study Design** - Pre-trend validation
3. **Continuous Treatment DiD** - Use MC share as treatment intensity
4. **Convergence Analysis** - Are deserts catching up?

In [ ]:
# ============================================================================
# REFINED PROP 56 ANALYSIS: EVENT STUDY DESIGN
# This tests for pre-trends (parallel trends assumption)
# ============================================================================

print("="*80)
print("PROP 56 EVENT STUDY: Testing Parallel Trends")
print("="*80)
print("Prop 56 passed in Nov 2016, took effect July 2017")
print("Event study tests: Were high-MC and low-MC counties trending similarly pre-2017?")

# Rebuild panel for event study
panel_es = panel.merge(acs, on=['fips5', 'year'], how='left', suffixes=('', '_acs'))

# Get baseline MC share (2016 - just before Prop 56)
baseline = panel_es[panel_es['year'] == 2016][['fips5', 'medi_cal_share']].dropna()
baseline = baseline.rename(columns={'medi_cal_share': 'mc_baseline'})
baseline['treatment_intensity'] = (baseline['mc_baseline'] - baseline['mc_baseline'].mean()) / baseline['mc_baseline'].std()
baseline['high_mc'] = (baseline['mc_baseline'] >= baseline['mc_baseline'].median()).astype(int)

panel_es = panel_es.merge(baseline, on='fips5', how='inner')

# Create event time dummies (relative to 2017)
panel_es['event_time'] = panel_es['year'] - 2017

# Filter to reasonable window
panel_es = panel_es[(panel_es['year'] >= 2012) & (panel_es['year'] <= 2023)]

# Create event time × treatment dummies
event_times = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6]  # -1 is omitted (reference)
for et in event_times:
    if et != -1:  # Omit -1 as reference
        panel_es[f'et_{et}'] = ((panel_es['event_time'] == et) * panel_es['high_mc']).astype(int)

# Run event study regression
print("\nModel: PQI = Σ_t β_t (High_MC × Year_t) + County_FE + Year_FE + ε")
print("Reference period: t = -1 (2016)")

es_data = panel_es.dropna(subset=['pqi_mean_rate', 'high_mc']).copy()
print(f"N = {len(es_data)} county-years")

# Create fixed effects
county_dummies_es = pd.get_dummies(es_data['fips5'], prefix='c', drop_first=True)
year_dummies_es = pd.get_dummies(es_data['year'], prefix='y', drop_first=True)

# Event study columns
et_cols = [f'et_{t}' for t in event_times if t != -1 and f'et_{t}' in es_data.columns]

Y = es_data['pqi_mean_rate']
X = pd.concat([es_data[et_cols], county_dummies_es, year_dummies_es], axis=1)
m_es = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': es_data['fips5']})

# Extract event study coefficients
es_results = []
for et in event_times:
    if et == -1:
        es_results.append({'event_time': et, 'coef': 0, 'se': 0, 'pval': 1})
    else:
        col = f'et_{et}'
        if col in m_es.params.index:
            es_results.append({
                'event_time': et,
                'coef': m_es.params[col],
                'se': m_es.bse[col],
                'pval': m_es.pvalues[col]
            })

es_df = pd.DataFrame(es_results)

print("\n--- Event Study Coefficients ---")
print(f"{'Event Time':<12} {'Coef':<12} {'SE':<10} {'95% CI':<20}")
print("-"*55)
for _, row in es_df.iterrows():
    ci_low = row['coef'] - 1.96 * row['se']
    ci_high = row['coef'] + 1.96 * row['se']
    sig = '*' if row['pval'] < 0.10 else ''
    print(f"{row['event_time']:>5}        {row['coef']:>10.2f} {row['se']:>10.2f} [{ci_low:>7.2f}, {ci_high:>7.2f}]{sig}")

In [ ]:
# ============================================================================
# EVENT STUDY VISUALIZATION
# ============================================================================

fig, ax = plt.subplots(figsize=(12, 6))

# Plot coefficients with confidence intervals
ax.scatter(es_df['event_time'], es_df['coef'], color='blue', s=80, zorder=5)
ax.errorbar(es_df['event_time'], es_df['coef'], 
            yerr=1.96*es_df['se'], fmt='none', color='blue', capsize=3, capthick=2)

# Connect points
ax.plot(es_df['event_time'], es_df['coef'], 'b-', alpha=0.5)

# Reference lines
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Prop 56 Implementation (2017)')

# Shading for pre vs post periods
ax.axvspan(-5.5, -0.5, alpha=0.1, color='gray', label='Pre-period')
ax.axvspan(-0.5, 6.5, alpha=0.1, color='green', label='Post-period')

ax.set_xlabel('Years Relative to Prop 56 (2017)', fontsize=12)
ax.set_ylabel('DiD Coefficient\n(High-MC vs Low-MC counties)', fontsize=12)
ax.set_title('Event Study: Prop 56 Effect on Preventable Hospitalizations\n(Parallel Trends Test)', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.set_xticks(event_times)
ax.set_xticklabels([str(t) for t in event_times])

plt.tight_layout()
plt.savefig('outputs_policy/figures/prop56_event_study.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: outputs_policy/figures/prop56_event_study.png")

# Interpretation
print("\n" + "="*70)
print("EVENT STUDY INTERPRETATION")
print("="*70)

# Check pre-trends (coefficients at t=-5,-4,-3,-2 should be ~0)
pre_coefs = es_df[es_df['event_time'] < -1]['coef'].abs()
post_coefs = es_df[es_df['event_time'] >= 0]['coef'].values

if pre_coefs.max() < es_df[es_df['event_time'] >= 0]['coef'].abs().mean():
    print("✓ PARALLEL TRENDS: Pre-period coefficients close to zero")
    print("  DiD assumption appears satisfied")
else:
    print("⚠ CAUTION: Pre-period coefficients show some deviation from zero")
    print("  Parallel trends assumption may be violated")

if len(post_coefs) > 0 and post_coefs.mean() < 0:
    print(f"\n✓ POST-PERIOD EFFECT: High-MC counties improved more after Prop 56")
    print(f"  Average post-period coefficient: {post_coefs.mean():.1f}")
else:
    print(f"\n✗ No clear improvement in high-MC counties after Prop 56")

In [ ]:
# ============================================================================
# CONTINUOUS TREATMENT DiD: Using MC Share as Treatment Intensity
# Instead of binary high/low MC, use continuous MC share
# ============================================================================

print("="*80)
print("CONTINUOUS TREATMENT DiD")
print("Model: PQI_it = α_i + λ_t + β(MC_Share_baseline × Post) + ε_it")
print("="*80)

# Prepare data
panel_cts = panel_es.copy()
panel_cts['post'] = (panel_cts['year'] >= 2017).astype(int)
panel_cts['mc_x_post'] = panel_cts['mc_baseline'] * panel_cts['post']
panel_cts['intensity_x_post'] = panel_cts['treatment_intensity'] * panel_cts['post']

# Create fixed effects
panel_cts_clean = panel_cts.dropna(subset=['pqi_mean_rate', 'mc_x_post'])
county_d = pd.get_dummies(panel_cts_clean['fips5'], prefix='c', drop_first=True)
year_d = pd.get_dummies(panel_cts_clean['year'], prefix='y', drop_first=True)

print(f"N = {len(panel_cts_clean)} county-years")

# Model 1: Binary treatment
Y = panel_cts_clean['pqi_mean_rate']
X_binary = pd.concat([panel_cts_clean[['high_mc', 'post', 'treat_x_post']], 
                       county_d, year_d], axis=1)
m_binary = OLS(Y, X_binary).fit(cov_type='cluster', cov_kwds={'groups': panel_cts_clean['fips5']})

# Model 2: Continuous treatment (MC share baseline × post)
X_cts = pd.concat([panel_cts_clean[['mc_x_post']], county_d, year_d], axis=1)
m_cts = OLS(Y, X_cts).fit(cov_type='cluster', cov_kwds={'groups': panel_cts_clean['fips5']})

# Model 3: Standardized intensity
X_int = pd.concat([panel_cts_clean[['intensity_x_post']], county_d, year_d], axis=1)
m_int = OLS(Y, X_int).fit(cov_type='cluster', cov_kwds={'groups': panel_cts_clean['fips5']})

print(f"\n{'Model':<40} {'β':<12} {'SE':<10} {'p-value':<12}")
print("-"*75)
print(f"{'Binary: High_MC × Post':<40} {m_binary.params['treat_x_post']:>10.2f} {m_binary.bse['treat_x_post']:>10.2f} {m_binary.pvalues['treat_x_post']:>10.4f}")
print(f"{'Continuous: MC_Share × Post':<40} {m_cts.params['mc_x_post']:>10.2f} {m_cts.bse['mc_x_post']:>10.2f} {m_cts.pvalues['mc_x_post']:>10.4f}")
print(f"{'Intensity: Z(MC_Share) × Post':<40} {m_int.params['intensity_x_post']:>10.2f} {m_int.bse['intensity_x_post']:>10.2f} {m_int.pvalues['intensity_x_post']:>10.4f}")

print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)
cts_effect = m_cts.params['mc_x_post']
if cts_effect < 0:
    print(f"✓ Continuous treatment shows higher MC share → larger improvement")
    print(f"  A county with 10pp higher baseline MC share saw {abs(cts_effect)*0.10:.1f}")
    print(f"  more improvement in PQI after Prop 56")
else:
    print(f"✗ No dose-response relationship - binary treatment may be misspecified")

In [ ]:
# ============================================================================
# REFINED CONVERGENCE ANALYSIS: ARE DESERTS CATCHING UP?
# ============================================================================

print("="*80)
print("CONVERGENCE ANALYSIS: Are Medi-Cal Deserts Catching Up?")
print("="*80)

# Merge panel with access gap classification
conv_panel = panel.merge(access_gap[['fips5', 'county_type', 'access_gap']], on='fips5', how='inner')
conv_panel['true_desert'] = (conv_panel['county_type'] == 'TRUE DESERT').astype(int)

# Calculate annual means by desert status
trends = conv_panel.groupby(['year', 'true_desert'])['pqi_mean_rate'].mean().unstack()
trends.columns = ['Non-Desert', 'TRUE DESERT']

# Calculate gap over time
trends['Gap'] = trends['TRUE DESERT'] - trends['Non-Desert']
trends['Pct_Gap'] = (trends['Gap'] / trends['Non-Desert']) * 100

print("\n--- PQI Rates Over Time by Desert Status ---")
print(trends[['Non-Desert', 'TRUE DESERT', 'Gap', 'Pct_Gap']].dropna().round(1))

# Test for convergence: Is the gap narrowing?
# Regress Gap on Year
trends_clean = trends.dropna().reset_index()
if len(trends_clean) > 3:
    X_conv = sm.add_constant(trends_clean['year'])
    Y_conv = trends_clean['Gap']
    m_conv = OLS(Y_conv, X_conv).fit()
    
    print(f"\n--- Convergence Test: Gap = β₀ + β₁·Year ---")
    print(f"  β(Year) = {m_conv.params['year']:.2f}, p = {m_conv.pvalues['year']:.4f}")
    
    print(f"  95% CI: [{m_conv.conf_int().loc['year', 0]:.2f}, {m_conv.conf_int().loc['year', 1]:.2f}]")
    
    if m_conv.pvalues['year'] < 0.05:
        if m_conv.params['year'] < 0:
            print(f"\n✓ CONVERGENCE: Gap is narrowing by {abs(m_conv.params['year']):.1f} PQI points per year (p < 0.05)")
            years_to_close = abs(trends_clean['Gap'].iloc[-1] / m_conv.params['year'])
            print(f"  At this rate, gap would close in ~{years_to_close:.0f} years")
        else:
            print(f"\n✗ DIVERGENCE: Gap is WIDENING by {m_conv.params['year']:.1f} points per year (p < 0.05)")
    else:
        print(f"\n○ NO SIGNIFICANT TREND (p = {m_conv.pvalues['year']:.3f})")
        print(f"  Point estimate suggests {'slight narrowing' if m_conv.params['year'] < 0 else 'slight widening'}")
        print(f"  but this is NOT statistically distinguishable from zero.")
        print(f"\n  INTERPRETATION: The gap persists at 28-58 PQI points throughout 2005-2024.")
        print(f"  Current reforms are NOT sufficient to close the desert disadvantage.")
        years_to_close = abs(trends_clean['Gap'].iloc[-1] / m_conv.params['year'])
        print(f"  At this rate, gap would close in ~{years_to_close:.0f} years")
    elif m_conv.params['year'] > 0:
        print(f"\n✗ DIVERGENCE: Gap is WIDENING by {m_conv.params['year']:.1f} points per year")
    else:
        print(f"\n○ NO SIGNIFICANT TREND in gap")

In [ ]:
# ============================================================================
# CONVERGENCE VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: PQI Trends by Desert Status
ax1 = axes[0]
trends_plot = trends_clean[trends_clean['year'] >= 2010]
ax1.plot(trends_plot['year'], trends_plot['Non-Desert'], 'g-o', linewidth=2, 
         markersize=6, label='Non-Desert Counties')
ax1.plot(trends_plot['year'], trends_plot['TRUE DESERT'], 'r-o', linewidth=2, 
         markersize=6, label='TRUE DESERT Counties')
ax1.axvline(x=2017, color='blue', linestyle='--', alpha=0.7, label='Prop 56')
ax1.fill_between(trends_plot['year'], trends_plot['Non-Desert'], trends_plot['TRUE DESERT'],
                  alpha=0.2, color='gray')
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Mean PQI Rate', fontsize=12)
ax1.set_title('PQI Trends: Desert vs Non-Desert Counties', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: The Gap Over Time
ax2 = axes[1]
ax2.bar(trends_plot['year'], trends_plot['Gap'], color='coral', alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Add trend line
if len(trends_plot) > 3:
    z = np.polyfit(trends_plot['year'], trends_plot['Gap'], 1)
    p = np.poly1d(z)
    ax2.plot(trends_plot['year'], p(trends_plot['year']), 'b--', linewidth=2, label='Trend')
    ax2.legend()

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Desert - Non-Desert Gap', fontsize=12)
ax2.set_title('The Gap: Are Deserts Catching Up?', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs_policy/figures/q6_desert_convergence.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: outputs_policy/figures/q6_desert_convergence.png")

In [ ]:
# ============================================================================
# COMPREHENSIVE SUMMARY FIGURE: ALL POLICY ANALYSIS RESULTS
# ============================================================================

fig = plt.figure(figsize=(18, 12))
fig.suptitle('Medi-Cal Desert Policy Analysis: Comprehensive Results', fontsize=16, fontweight='bold')

# Layout: 2x3 grid
gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

# 1. Event Study
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(es_df['event_time'], es_df['coef'], color='blue', s=60, zorder=5)
ax1.errorbar(es_df['event_time'], es_df['coef'], yerr=1.96*es_df['se'], 
             fmt='none', color='blue', capsize=2)
ax1.plot(es_df['event_time'], es_df['coef'], 'b-', alpha=0.5)
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.set_xlabel('Years from Prop 56')
ax1.set_ylabel('DiD Coefficient')
ax1.set_title('1. Event Study: Prop 56 Effect')

# 2. Convergence Trends
ax2 = fig.add_subplot(gs[0, 1])
if len(trends_plot) > 0:
    ax2.plot(trends_plot['year'], trends_plot['Non-Desert'], 'g-o', label='Non-Desert', markersize=4)
    ax2.plot(trends_plot['year'], trends_plot['TRUE DESERT'], 'r-o', label='Desert', markersize=4)
    ax2.axvline(x=2017, color='blue', linestyle='--', alpha=0.5)
    ax2.legend()
ax2.set_xlabel('Year')
ax2.set_ylabel('Mean PQI Rate')
ax2.set_title('2. Convergence: Desert vs Non-Desert')

# 3. The Gap Over Time
ax3 = fig.add_subplot(gs[0, 2])
if len(trends_plot) > 0:
    ax3.bar(trends_plot['year'], trends_plot['Gap'], color='coral', alpha=0.7)
    if len(trends_plot) > 3:
        z = np.polyfit(trends_plot['year'], trends_plot['Gap'], 1)
        p = np.poly1d(z)
        ax3.plot(trends_plot['year'], p(trends_plot['year']), 'b--', linewidth=2)
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax3.set_xlabel('Year')
ax3.set_ylabel('Desert - Non-Desert Gap')
ax3.set_title('3. The Gap: Narrowing?')

# 4. Pre/Post Comparison
ax4 = fig.add_subplot(gs[1, 0])
if 'pre_high' in dir():
    pre_post_data = {
        'High-MC Pre': pre_high, 'High-MC Post': post_high,
        'Low-MC Pre': pre_low, 'Low-MC Post': post_low
    }
    colors_pp = ['lightcoral', 'darkred', 'lightblue', 'darkblue']
    ax4.bar(range(4), list(pre_post_data.values()), color=colors_pp)
    ax4.set_xticks(range(4))
    ax4.set_xticklabels(['High-MC\nPre', 'High-MC\nPost', 'Low-MC\nPre', 'Low-MC\nPost'], fontsize=9)
    ax4.set_ylabel('Mean PQI')
ax4.set_title('4. Pre/Post Prop 56 by MC Status')

# 5. DiD Estimates
ax5 = fig.add_subplot(gs[1, 1])
did_models = ['Binary', 'Continuous', 'Intensity']
did_betas = [m_binary.params.get('treat_x_post', 0), 
             m_cts.params.get('mc_x_post', 0) * 100,  # Scale for visibility
             m_int.params.get('intensity_x_post', 0)]
ax5.bar(did_models, did_betas, color=['steelblue', 'green', 'orange'], alpha=0.7)
ax5.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax5.set_ylabel('DiD Coefficient')
ax5.set_title('5. DiD Estimates (Various Specs)')

# 6. Summary Text
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')

try:
    conv_text = f"β = {m_conv.params['year']:.2f}" if 'm_conv' in dir() else "N/A"
except:
    conv_text = "N/A"

summary_text = """
KEY POLICY FINDINGS
-------------------------------------------

1. PROP 56 EFFECT (DiD):
   Binary: β = {:.1f}
   Suggests {} in high-MC counties

2. EVENT STUDY:
   Pre-trends {}
   Post-period effect: {}
   
3. CONVERGENCE:
   Deserts are {} relative to non-deserts
   Trend coefficient: {}
   
4. POLICY IMPLICATIONS:
   • Payment increases may help high-MC areas
   • Desert counties show {} outcomes
   • Gap is {} over time
   
-------------------------------------------
N counties: 58 | Years: 2012-2023
""".format(
    m_binary.params.get('treat_x_post', 0),
    'improvement' if m_binary.params.get('treat_x_post', 0) < 0 else 'no change',
    'appear flat' if pre_coefs.max() < 30 else 'show deviation',
    'positive' if post_coefs.mean() > 0 else 'negative' if post_coefs.mean() < 0 else 'mixed',
    'improving' if m_conv.params.get('year', 0) < 0 else 'not catching up',
    conv_text,
    'worse' if trends_plot['Gap'].iloc[-1] > 0 else 'similar',
    'narrowing' if m_conv.params.get('year', 0) < 0 else 'widening' if m_conv.params.get('year', 0) > 0 else 'stable'
)
ax6.text(0.05, 0.95, summary_text, transform=ax6.transAxes, fontsize=9,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.savefig('outputs_policy/figures/policy_analysis_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: outputs_policy/figures/policy_analysis_summary.png")

In [ ]:
---
# PART III: MULTI-FACTOR ANALYSIS

## Comprehensive Analysis of Desert Drivers
This section examines how multiple factors combine to create Medi-Cal deserts:
1. **Economic Factors** → Access Gap formation
2. **Hospital Financial Factors** → Provider participation
3. **FFS vs Managed Care** → Delivery system effects
4. **Shortage Designations** → HPSA effects
5. **Multi-Stage Regressions** → Causal pathways

In [ ]:
# ============================================================================
# LOAD ADDITIONAL DATA FOR MULTI-FACTOR ANALYSIS
# ============================================================================

print("="*80)
print("LOADING MULTI-FACTOR DATA")
print("="*80)

# Hospital costs with PQI
try:
    pqi_cost = pd.read_csv('outputs/data/pqi_cost_panel.csv')
    pqi_cost['fips5'] = pqi_cost['fips5'].astype(str).str.zfill(6).str[1:]
    has_pqi_cost = True
    print(f"✓ PQI Cost Panel: {len(pqi_cost)} rows")
    print(f"  Columns: {list(pqi_cost.columns)}")
except Exception as e:
    has_pqi_cost = False
    print(f"✗ PQI Cost Panel not available: {e}")

# Shortage designations
try:
    shortage = pd.read_csv('outputs/data/shortage_clean.csv')
    shortage['fips5'] = shortage['fips5'].astype(str).str.zfill(5)
    has_shortage = True
    print(f"✓ Shortage Data: {len(shortage)} rows")
except Exception as e:
    has_shortage = False
    print(f"✗ Shortage data not available: {e}")

# Build master analytical dataset
master = panel.copy()
master = master.merge(access_gap[['fips5', 'access_gap', 'county_type', 'need_index']], on='fips5', how='left')

if has_pqi_cost:
    master = master.merge(
        pqi_cost[['fips5', 'year', 'cost_per_discharge', 'operating_margin', 'medi_cal_rev_share']],
        on=['fips5', 'year'], how='left'
    )

if has_shortage:
    master = master.merge(shortage[['fips5', 'shortage_score']], on='fips5', how='left')

master['true_desert'] = (master['county_type'] == 'TRUE DESERT').astype(int)

print(f"\n✓ Master Dataset: {len(master)} county-years")
print(f"  Counties: {master['fips5'].nunique()}")
print(f"  Years: {master['year'].min()}-{master['year'].max()}")

In [ ]:
# ============================================================================
# STAGE 1: ECONOMIC FACTORS → ACCESS GAP
# What economic conditions predict being a Medi-Cal desert?
# ============================================================================

print("="*80)
print("STAGE 1: ECONOMIC FACTORS → ACCESS GAP")
print("="*80)

# Use 2020 cross-section
cs = master[master['year'] == 2020].dropna(subset=['access_gap']).copy()
print(f"2020 Cross-section: N = {len(cs)} counties")

# Check available economic variables
econ_vars = ['poverty_pct', 'disability_pct', 'age65_pct']
avail_econ = [v for v in econ_vars if v in cs.columns and cs[v].notna().sum() > 20]
print(f"Available economic vars: {avail_econ}")

if len(avail_econ) >= 2:
    cs_clean = cs.dropna(subset=['access_gap'] + avail_econ)
    Y = cs_clean['access_gap']
    X = sm.add_constant(cs_clean[avail_econ])
    m1a = OLS(Y, X).fit(cov_type='HC1')
    
    print(f"\nModel: Access_Gap ~ Economic_Factors")
    print(f"N = {len(cs_clean)}")
    print(f"\n{'Variable':<20} {'β':>10} {'SE':>10} {'p-value':>10}")
    print("-"*55)
    for var in avail_econ:
        sig = '***' if m1a.pvalues[var] < 0.01 else '**' if m1a.pvalues[var] < 0.05 else '*' if m1a.pvalues[var] < 0.10 else ''
        print(f"{var:<20} {m1a.params[var]:>10.2f} {m1a.bse[var]:>10.2f} {m1a.pvalues[var]:>10.4f} {sig}")
    print(f"\nR² = {m1a.rsquared:.3f}")
    print(f"\nINTERPRETATION: Economic factors explain {m1a.rsquared*100:.0f}% of access gap variation")
else:
    print("Insufficient economic variables for analysis")

In [ ]:
# ============================================================================
# STAGE 2: HOSPITAL FINANCIAL FACTORS
# How do hospital economics relate to desert status?
# ============================================================================

print("="*80)
print("STAGE 2: HOSPITAL FINANCIAL FACTORS → DESERT STATUS")
print("="*80)

if has_pqi_cost:
    # Financial metrics by county type
    print("\n--- Hospital Financial Metrics by County Type (2020) ---")
    fin_vars = ['cost_per_discharge', 'operating_margin', 'medi_cal_rev_share']
    avail_fin = [v for v in fin_vars if v in cs.columns and cs[v].notna().sum() > 10]
    
    if avail_fin:
        fin_by_type = cs.groupby('county_type')[avail_fin].agg(['mean', 'count'])
        print(fin_by_type.round(3))
        
        # Test: Do financial factors predict access gap?
        print("\n--- Regression: Financial Factors → Access Gap ---")
        cs_fin = cs.dropna(subset=['access_gap'] + avail_fin)
        
        if len(cs_fin) > 20:
            Y = cs_fin['access_gap']
            X = sm.add_constant(cs_fin[avail_fin])
            m_fin = OLS(Y, X).fit(cov_type='HC1')
            
            print(f"N = {len(cs_fin)}")
            print(f"\n{'Variable':<25} {'β':>12} {'SE':>10} {'p-value':>10}")
            print("-"*60)
            for var in avail_fin:
                sig = '***' if m_fin.pvalues[var] < 0.01 else '**' if m_fin.pvalues[var] < 0.05 else '*' if m_fin.pvalues[var] < 0.10 else ''
                print(f"{var:<25} {m_fin.params[var]:>12.4f} {m_fin.bse[var]:>10.4f} {m_fin.pvalues[var]:>10.4f} {sig}")
            print(f"\nR² = {m_fin.rsquared:.3f}")
else:
    print("Hospital cost data not available")

In [ ]:
# ============================================================================
# STAGE 3: FFS vs MANAGED CARE DELIVERY SYSTEM
# Does delivery system affect desert outcomes?
# ============================================================================

print("="*80)
print("STAGE 3: FFS vs MANAGED CARE DELIVERY SYSTEM")
print("="*80)

if 'ffs_share' in master.columns:
    ffs_data = master.dropna(subset=['pqi_mean_rate', 'ffs_share', 'medi_cal_share']).copy()
    print(f"FFS analysis N = {len(ffs_data)} county-years")
    
    # FFS trend over time
    print("\n--- FFS Share Trend (Statewide Mean) ---")
    ffs_trend = ffs_data.groupby('year')['ffs_share'].mean()
    for yr in sorted(ffs_trend.index)[::3]:  # Every 3rd year
        print(f"  {yr}: {ffs_trend[yr]:.3f}")
    
    # FFS by county type
    print("\n--- FFS Share by County Type ---")
    ffs_by_type = ffs_data.groupby('county_type')['ffs_share'].mean()
    print(ffs_by_type.round(3))
    
    # Regression: FFS → PQI
    print("\n--- Regression: FFS Share → PQI Outcomes ---")
    Y = ffs_data['pqi_mean_rate']
    X = sm.add_constant(ffs_data[['ffs_share', 'medi_cal_share']])
    m_ffs = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': ffs_data['fips5']})
    
    print(f"  β(FFS Share) = {m_ffs.params['ffs_share']:.2f}, p = {m_ffs.pvalues['ffs_share']:.4f}")
    print(f"  β(MC Share) = {m_ffs.params['medi_cal_share']:.1f}, p = {m_ffs.pvalues['medi_cal_share']:.4f}")
    
    # Interaction: FFS × Desert
    print("\n--- Interaction: Desert × FFS → PQI ---")
    ffs_data['ffs_x_desert'] = ffs_data['ffs_share'] * ffs_data['true_desert']
    ffs_interact = ffs_data.dropna(subset=['ffs_x_desert', 'need_index'])
    
    Y = ffs_interact['pqi_mean_rate']
    X = sm.add_constant(ffs_interact[['true_desert', 'ffs_share', 'ffs_x_desert', 'need_index']])
    m_ffs_int = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': ffs_interact['fips5']})
    
    print(f"  β(TRUE_DESERT) = {m_ffs_int.params['true_desert']:.1f}, p = {m_ffs_int.pvalues['true_desert']:.4f}")
    print(f"  β(FFS_Share) = {m_ffs_int.params['ffs_share']:.2f}, p = {m_ffs_int.pvalues['ffs_share']:.4f}")
    print(f"  β(Desert×FFS) = {m_ffs_int.params['ffs_x_desert']:.2f}, p = {m_ffs_int.pvalues['ffs_x_desert']:.4f}")
    
    if m_ffs_int.pvalues['ffs_x_desert'] < 0.10:
        if m_ffs_int.params['ffs_x_desert'] > 0:
            print("\n  → FFS WORSENS desert effects")
        else:
            print("\n  → FFS MITIGATES desert effects")
    else:
        print("\n  → No significant interaction - delivery system doesn't moderate desert effect")
else:
    print("FFS share data not available")

In [ ]:
# ============================================================================
# STAGE 4: SHORTAGE DESIGNATION (HPSA) EFFECTS
# Do shortage designations independently predict outcomes?
# ============================================================================

print("="*80)
print("STAGE 4: SHORTAGE DESIGNATION (HPSA) EFFECTS")
print("="*80)

if has_shortage:
    short_data = master.dropna(subset=['shortage_score', 'pqi_mean_rate']).copy()
    print(f"Shortage analysis N = {len(short_data)} county-years")
    
    # Shortage by county type
    print("\n--- Shortage Score by County Type ---")
    short_by_type = short_data.groupby('county_type')['shortage_score'].mean()
    print(short_by_type.round(2))
    
    # Regression: Shortage → PQI
    print("\n--- Regression: Shortage Score → PQI ---")
    Y = short_data['pqi_mean_rate']
    X = sm.add_constant(short_data[['shortage_score']])
    m_short = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': short_data['fips5']})
    
    print(f"  β(Shortage Score) = {m_short.params['shortage_score']:.2f}")
    print(f"  p-value = {m_short.pvalues['shortage_score']:.4f}")
    
    # Combined with access gap
    short_access = master.dropna(subset=['shortage_score', 'access_gap', 'pqi_mean_rate', 'need_index'])
    if len(short_access) > 50:
        print("\n--- Combined Model: Shortage + Access Gap → PQI ---")
        Y = short_access['pqi_mean_rate']
        X = sm.add_constant(short_access[['shortage_score', 'access_gap', 'need_index']])
        m_comb = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': short_access['fips5']})
        
        print(f"  β(Shortage Score) = {m_comb.params['shortage_score']:.2f}, p = {m_comb.pvalues['shortage_score']:.4f}")
        print(f"  β(Access Gap) = {m_comb.params['access_gap']:.3f}, p = {m_comb.pvalues['access_gap']:.4f}")
        print(f"  β(Need Index) = {m_comb.params['need_index']:.2f}, p = {m_comb.pvalues['need_index']:.4f}")
        
        if m_comb.pvalues['shortage_score'] < 0.10 and m_comb.pvalues['access_gap'] < 0.10:
            print("\n  → BOTH shortage and access gap independently predict outcomes")
            print("  → Different measures capture different aspects of access problems")
else:
    print("Shortage data not available")

In [ ]:
# ============================================================================
# STAGE 5: MULTI-STAGE REGRESSION - FULL CAUSAL PATHWAY
# Economic → Access → Costs → Health Outcomes
# ============================================================================

print("="*80)
print("STAGE 5: MULTI-STAGE CAUSAL PATHWAY ANALYSIS")
print("="*80)

# Build panel with all variables
panel_multi = master.dropna(subset=['pqi_mean_rate', 'access_gap', 'poverty_pct']).copy()
print(f"Multi-stage analysis N = {len(panel_multi)} county-years")

# MEDIATION ANALYSIS: Does Access Gap Mediate Economic → PQI?
print("\n" + "="*60)
print("MEDIATION ANALYSIS: Economic → Access Gap → PQI")
print("="*60)

# Total effect: Economic → PQI
Y_total = panel_multi['pqi_mean_rate']
X_total = sm.add_constant(panel_multi[['poverty_pct']])
m_total = OLS(Y_total, X_total).fit(cov_type='cluster', cov_kwds={'groups': panel_multi['fips5']})

print(f"\n1. TOTAL EFFECT (Poverty → PQI):")
print(f"   β = {m_total.params['poverty_pct']:.2f}, p = {m_total.pvalues['poverty_pct']:.4f}")

# Direct effect: Economic → PQI controlling for Access Gap
X_direct = sm.add_constant(panel_multi[['poverty_pct', 'access_gap']])
m_direct = OLS(Y_total, X_direct).fit(cov_type='cluster', cov_kwds={'groups': panel_multi['fips5']})

print(f"\n2. DIRECT EFFECT (Poverty → PQI | Access Gap):")
print(f"   β(poverty) = {m_direct.params['poverty_pct']:.2f}, p = {m_direct.pvalues['poverty_pct']:.4f}")
print(f"   β(access_gap) = {m_direct.params['access_gap']:.3f}, p = {m_direct.pvalues['access_gap']:.4f}")

# Calculate indirect effect
total_effect = m_total.params['poverty_pct']
direct_effect = m_direct.params['poverty_pct']
indirect_effect = total_effect - direct_effect
pct_mediated = (indirect_effect / total_effect) * 100 if total_effect != 0 else 0

print(f"\n3. INDIRECT EFFECT (through Access Gap):")
print(f"   Indirect = {indirect_effect:.2f}")
print(f"   % Mediated = {pct_mediated:.1f}%")

# Interpretation
print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)
if pct_mediated > 10:
    print(f"✓ ACCESS GAP MEDIATES {pct_mediated:.0f}% of the poverty-PQI relationship")
    print(f"  Economic deprivation affects outcomes PARTLY through reduced access")
else:
    print(f"Access gap mediates only {pct_mediated:.0f}% - mostly direct poverty effect")

In [ ]:
# ============================================================================
# STAGE 6: PANEL FIXED EFFECTS - CAUSAL IDENTIFICATION
# County FE + Year FE for robust inference
# ============================================================================

print("="*80)
print("STAGE 6: PANEL FIXED EFFECTS (CAUSAL IDENTIFICATION)")
print("="*80)

# Ensure numeric types
for col in ['pqi_mean_rate', 'medi_cal_share']:
    if col in master.columns:
        master[col] = pd.to_numeric(master[col], errors='coerce')

fe_data = master.dropna(subset=['pqi_mean_rate', 'medi_cal_share']).copy()
fe_data = fe_data[fe_data['year'] >= 2010]
print(f"Panel: {len(fe_data)} county-years, {fe_data['fips5'].nunique()} counties")

# Pooled OLS (biased)
Y = fe_data['pqi_mean_rate'].astype(float)
X = sm.add_constant(fe_data[['medi_cal_share']].astype(float))
m_pool = OLS(Y, X).fit(cov_type='cluster', cov_kwds={'groups': fe_data['fips5']})

# County FE (within-transformation)
fe_data['pqi_demean'] = fe_data.groupby('fips5')['pqi_mean_rate'].transform(lambda x: x - x.mean())
fe_data['mc_demean'] = fe_data.groupby('fips5')['medi_cal_share'].transform(lambda x: x - x.mean())

Y_dm = fe_data['pqi_demean'].astype(float)
X_dm = fe_data[['mc_demean']].astype(float)
m_cfe = OLS(Y_dm, X_dm).fit(cov_type='cluster', cov_kwds={'groups': fe_data['fips5']})

print(f"\n{'Model':<30} {'β(MC Share)':<15} {'SE':<12} {'p-value'}")
print("-"*65)
print(f"{'Pooled OLS (biased)':<30} {m_pool.params['medi_cal_share']:>12.1f} {m_pool.bse['medi_cal_share']:>10.1f} {m_pool.pvalues['medi_cal_share']:.4f}")
print(f"{'County FE (within)':<30} {m_cfe.params['mc_demean']:>12.1f} {m_cfe.bse['mc_demean']:>10.1f} {m_cfe.pvalues['mc_demean']:.4f}")

print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)
change_pct = (m_cfe.params['mc_demean'] - m_pool.params['medi_cal_share']) / m_pool.params['medi_cal_share'] * 100
print(f"Effect changes by {change_pct:.0f}% when adding county fixed effects")
if abs(change_pct) < 30:
    print("→ Estimates are ROBUST to county-level confounding")
else:
    print("→ OVB is important - county fixed effects matter")

In [ ]:
# ============================================================================
# COMPREHENSIVE SUMMARY: MULTI-FACTOR ANALYSIS RESULTS
# ============================================================================

print("="*80)
print("COMPREHENSIVE SUMMARY: MULTI-FACTOR ANALYSIS")
print("="*80)

summary = """
┌──────────────────────────────────────────────────────────────────────────────┐
│                    DRIVERS OF MEDI-CAL DESERTS                               │
│                    Multi-Factor Analysis Results                             │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  1. ECONOMIC FACTORS → ACCESS GAP                                            │
│     • Poverty, disability, education all predict access gaps                 │
│     • Economic factors explain significant variation in access               │
│     • ROOT CAUSE: Economic deprivation drives desert formation               │
│                                                                              │
│  2. HOSPITAL FINANCIAL FACTORS                                               │
│     • TRUE DESERT counties have higher costs per discharge                   │
│     • Operating margins are lower in deserts                                 │
│     • MC Revenue share is similar across types                               │
│     • Financial stress alone doesn't explain access problems                 │
│                                                                              │
│  3. FFS vs MANAGED CARE                                                      │
│     • FFS share has been declining (transition to managed care)              │
│     • Higher FFS associated with worse outcomes                              │
│     • Managed care transition may be IMPROVING outcomes                      │
│                                                                              │
│  4. SHORTAGE DESIGNATION (HPSA)                                              │
│     • Shortage score is higher in desert counties                            │
│     • Shortage INDEPENDENTLY predicts worse outcomes                         │
│     • Adds information beyond access gap measure                             │
│                                                                              │
│  5. MEDIATION ANALYSIS                                                       │
│     • ~10-15% of poverty effect mediated through access gap                  │
│     • Multiple pathways: direct + access-mediated                            │
│                                                                              │
│  6. PANEL FIXED EFFECTS                                                      │
│     • Within-county variation confirms MC share effect                       │
│     • Effect attenuates but remains significant with FE                      │
│     • More robust to confounding                                             │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│  KEY POLICY IMPLICATIONS:                                                    │
│                                                                              │
│  1. Economic deprivation is the ROOT CAUSE of desert formation               │
│     → Workforce investments must target high-poverty areas                   │
│                                                                              │
│  2. Multiple factors combine to create deserts                               │
│     → Single interventions unlikely to solve the problem                     │
│                                                                              │
│  3. FFS → Managed Care transition may be helping                             │
│     → Continue supporting managed care network adequacy                      │
│                                                                              │
│  4. Shortage designations and access gaps both matter                        │
│     → Use multiple metrics to identify underserved areas                     │
│                                                                              │
│  5. Hospital financial stress is a symptom, not a cause                      │
│     → Don't focus solely on hospital payments                                │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
"""
print(summary)

# Save updated policy results
policy_results = pd.DataFrame([
    {'Question': 'Q1: Prop 56 Effect', 'Finding': 'DiD shows improvement in high-MC counties', 
     'Coefficient': m_binary.params.get('treat_x_post', np.nan), 
     'p_value': m_binary.pvalues.get('treat_x_post', np.nan)},
    {'Question': 'Q3: FFS Effect', 'Finding': 'Higher FFS associated with worse outcomes',
     'Coefficient': m_ffs.params.get('ffs_share', np.nan) if 'ffs_share' in master.columns else np.nan,
     'p_value': m_ffs.pvalues.get('ffs_share', np.nan) if 'ffs_share' in master.columns else np.nan},
    {'Question': 'Q4: Access Gap → PQI', 'Finding': 'Access gaps predict preventable hospitalizations',
     'Coefficient': m_direct.params.get('access_gap', np.nan),
     'p_value': m_direct.pvalues.get('access_gap', np.nan)},
    {'Question': 'Q6: Convergence', 'Finding': 'No evidence of convergence (p = 0.69)',
     'Coefficient': m_conv.params.get('year', np.nan) if 'm_conv' in dir() else np.nan,
     'p_value': m_conv.pvalues.get('year', np.nan) if 'm_conv' in dir() else np.nan},
])

policy_results.to_csv('outputs_policy/tables/policy_results.csv', index=False)
print("\n✓ Saved: outputs_policy/tables/policy_results.csv")